# Process calcium imaging data with DataJoint Elements

This notebook will walk through processing two-photon calcium imaging data collected
from ScanImage and processed with Suite2p.

The DataJoint Python API and Element Calcium Imaging offer a lot of features to support collaboration, automation, reproducibility, and visualization.

For more information on these topics, please visit our documentation: 
 
- [DataJoint Core](https://datajoint.com/docs/core/): General principles

- DataJoint [Python](https://datajoint.com/docs/core/datajoint-python/) and
  [MATLAB](https://datajoint.com/docs/core/datajoint-matlab/) APIs: in-depth reviews of
  specifics

- [DataJoint Element Calcium Imaging](https://datajoint.com/docs/elements/element-calcium-imaging/):
  A modular pipeline for calcium imaging analysis

In [ ]:
import datajoint as dj
import os

In [ ]:
dj.conn()

## Define schema prefixes and root data directory

+ If we set prefix to `neuro_`, every schema created with the current workflow will start with `neuro_`, e.g. `neuro_lab`, `neuro_scan`, `neuro_imaging`, etc.

+ The example dataset is mounted to this GitHub Codespace environment. Feel free to explore it in the `example_data` directory.

In [ ]:
dj.config["custom"] = {"database.prefix": "neuro_",
                       "imaging_root_data_dir": os.getenv('DJ_PUBLIC_S3_MOUNT_PATH', 'example_data')}

## Activate DataJoint Elements

+ The current workflow is composed of multiple database schemas, each of them corresponds to a module within `workflow_calcium_imaging.pipeline`

In [ ]:
from workflow_calcium_imaging.pipeline import lab, subject, session, scan, imaging, Equipment

## Workflow diagram

This workflow is assembled from 4 DataJoint elements:
+ [element-lab](https://github.com/datajoint/element-lab)
+ [element-animal](https://github.com/datajoint/element-animal)
+ [element-session](https://github.com/datajoint/element-session)
+ [element-calcium-imaging](https://github.com/datajoint/element-calcium-imaging)


In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(scan)
    + dj.Diagram(imaging)
)

## Insert entries into manual tables and populate automated tables

In [ ]:
subject.Subject.insert1(
    dict(
        subject="subject1",
        sex="F",
        subject_birth_date="2020-01-01",
        subject_description="ScanImage acquisition. Suite2p processing.",
    )
)

In [ ]:
Equipment.insert1(dict(scanner="ScanImage"))

In [ ]:
session_key = dict(subject="subject1", session_datetime="2021-04-30 12:22:15.032")

session.Session.insert1(session_key)

In [ ]:
session.SessionDirectory.insert1(
    dict(**session_key,
        session_dir="subject1/session1")
)

In [ ]:
scan.Scan.insert1(
    dict(
        **session_key,
        scan_id=0,
        scanner="ScanImage",
        acq_software="ScanImage",
        scan_notes="",
    )
)

In [ ]:
populate_settings = {
                    "display_progress": True,
                    "reserve_jobs": False,
                    "suppress_errors": False,
                    }

In [ ]:
# duration depends on your network bandwidth to s3
scan.ScanInfo.populate(**populate_settings)

In [ ]:
import suite2p

params_suite2p = suite2p.default_ops()
params_suite2p['nonrigid']=False

imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=params_suite2p,
    paramset_desc="Calcium imaging analysis with Suite2p using default Suite2p parameters",
)

In [ ]:
imaging.ProcessingTask.insert1(
    dict(
        **session_key,
        scan_id=0,
        paramset_idx=0,
        task_mode='load', # load or trigger
        processing_output_dir="subject1/session1/suite2p",
    )
)

In [ ]:
imaging.Processing.populate(**populate_settings)

In [ ]:
imaging.Curation.insert1(
    dict(
        **session_key,
        scan_id=0,
        paramset_idx=0,
        curation_id=0,
        curation_time="2021-04-30 12:22:15.032",
        curation_output_dir="subject1/session1/suite2p",
        manual_curation=False,
        curation_note="",
    )
)

In [ ]:
imaging.MotionCorrection.populate(**populate_settings)

In [ ]:
imaging.Segmentation.populate(**populate_settings)

In [ ]:
imaging.Fluorescence.populate(**populate_settings)

In [ ]:
imaging.Activity.populate(**populate_settings)

## Query, fetch, and plot segmentations on average image

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
average_image = (
    imaging.MotionCorrection.Summary & session_key & "field_idx=0"
).fetch1("average_image")

In [ ]:
mask_xpix, mask_ypix = (
    imaging.Segmentation.Mask * imaging.MaskClassification.MaskType
    & session_key
    & "mask_center_z=0"
    & "mask_npix > 130"
).fetch("mask_xpix", "mask_ypix")

In [ ]:
mask_image = np.zeros(np.shape(average_image), dtype=bool)
for xpix, ypix in zip(mask_xpix, mask_ypix):
    mask_image[ypix, xpix] = True

In [ ]:
plt.imshow(average_image)
plt.contour(mask_image, colors="white", linewidths=0.5);

# Drop schemas

+ Schemas are not typically dropped in a production workflow with real data in it. 
+ At the developmental phase, it might be required for the table redesign.
+ When dropping all schemas is needed, the following is the dependency order.

In [ ]:
def drop_databases(databases):
    import pymysql.err
    conn = dj.conn()

    with dj.config(safemode=False):
        for database in databases:
            schema = dj.Schema(f'{dj.config["custom"]["database.prefix"]}{database}')
            while schema.list_tables():
                for table in schema.list_tables():
                    try:
                        conn.query(f"DROP TABLE `{schema.database}`.`{table}`")
                    except pymysql.err.OperationalError:
                        print(f"Can't drop `{schema.database}`.`{table}`. Retrying...")
            schema.drop()

# drop_databases(databases=['imaging_report', 'imaging', 'scan', 'session', 'subject', 'lab', 'reference'])